# Imports

In [1]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import (RandomTreesEmbedding, IsolationForest, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder    

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Definisions

In [2]:
dataPrefix = '/home/data/aaia17/'
def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)        
def getNewForIndexes(dataFrame, names):
    X_selectedFeature = np.empty((len(dataFrame), 0), float)
    for name in names:
        X_selectedFeature = np.column_stack([X_selectedFeature, dataFrame[name].values])
    return X_selectedFeature
def dataScaling(data, scaler):
    data = data/(data+1)
    return scaler.fit_transform(data)

# Data reading from all chunks and making one  dataset

In [ ]:
trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)

testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)

In [ ]:
print(rawTrainingDataset.groupby('decision').size())

# Hand chosen features

In [ ]:
# Actually, we don't use it !

names = ['opponent.hp', 'opponent.played_minions_count', 'player.armor','player.hp', 'player.hand_count', 
         'player.played_minions_count', 'opponent.played.nOfCards', 'opponent.played.attack', 'opponent.played.crystals_cost'
         ,'opponent.played.hp_current', 'opponent.played.hp_max', 'player.played.nOfCards', 'player.played.attack',
        'player.played.crystals_cost', 'player.played.hp_current', 'player.played.hp_max', 'player.hand.nOfCards']
X_train_handChosen = getNewForIndexes(rawTrainingDataset, names)
X_test_handChosen  = getNewForIndexes(rawTestDataset, names)

#  Outliers detection and removing

In [ ]:
%%time
# Actually, we don't use it !

YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]

rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=10000, random_state=rng, n_jobs=-1)
clf.fit(XRawTrainingDataset)

trainingOutliers = clf.predict(XRawTrainingDataset)

XTrainigDatasetWithoutOutliers = XRawTrainingDataset[trainingOutliers>0]
YTrainingDatasetWithoutOutliers = YRawTrainingDataset[trainingOutliers>0]

# Automatic feature selection 

In [ ]:
%%time
# Actually, we don't use it !

estimator = LogisticRegression(random_state=rng)
selector = RFECV(estimator, step=1, cv=10, scoring='roc_auc', n_jobs=-1)
XTrainingSelected = selector.fit_transform(Xtry, Ytry)
pickle.dump(XTrainingSelected, open(dataPrefix+'XTrainingSelected.store',"wb"))

selector.ranking_

# Data scaling

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

XTrainingDatasetScaledWithOutliers = dataScaling(XRawTrainingDataset, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)

# Creating train and validation probes

In [ ]:
validation_size = 0.30
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(XTrainingDatasetScaledWithOutliers, 
        YRawTrainingDataset, test_size=validation_size, random_state=seed)

# Models probing

In [ ]:
# Actually, we don't use it !
%%time

seed = 7
scoring = 'roc_auc'
models = []
models.append(('RFC', RandomForestClassifier(n_jobs=-1, n_estimators=40)))

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

#  Main Model training

In [ ]:
%%time
clf = GradientBoostingClassifier(n_estimators=1000, max_depth=4)
clf = clf.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)
predictedTestWon = clf.predict_proba(XTestDatasetScaled)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted22.txt')